In [48]:
import git
import pandas as pd
import os
from datetime import datetime

In [25]:
git_root = git.Repo(os.getcwd(), search_parent_directories=True).git.rev_parse("--show-toplevel")
read_file_path = git_root + '/data/raw_data/understat_data.csv'

In [26]:
data = pd.read_csv(read_file_path, header=0, delimiter=';')

# Get first impression of dataset

In [33]:
len(data)

20980

In [29]:
data.columns

Index(['id', 'league', 'date', 'team_home', 'team_away', 'chances', 'goals',
       'xG', 'shots', 'shots_on_target', 'deep', 'ppda', 'xPTS'],
      dtype='object')

In [30]:
data.head()

,id,league,date,team_home,team_away,chances,goals,xG,shots,shots_on_target,deep,ppda,xPTS
0,81,EPL2015,Aug 08 2015,Manchester United,Tottenham,"['29', '39', '32']","['1', '0']","['0.63', '0.67']","['9', '9']","['1', '4']","['4', '10']","['13.83', '8.22']","['1.25', '1.36']"
1,82,EPL2015,Aug 08 2015,Bournemouth,Aston Villa,"['34', '36', '30']","['0', '1']","['0.88', '0.78']","['11', '7']","['2', '3']","['11', '2']","['6.90', '11.85']","['1.39', '1.25']"
2,83,EPL2015,Aug 08 2015,Everton,Watford,"['30', '43', '27']","['2', '2']","['0.60', '0.56']","['10', '11']","['5', '5']","['5', '4']","['6.65', '17.16']","['1.33', '1.24']"
3,84,EPL2015,Aug 08 2015,Leicester,Sunderland,"['64', '21', '15']","['4', '2']","['2.57', '1.46']","['19', '11']","['8', '5']","['5', '6']","['10.88', '9.56']","['2.13', '0.66']"
4,85,EPL2015,Aug 08 2015,Norwich,Crystal Palace,"['15', '21', '64']","['1', '3']","['1.13', '2.11']","['17', '11']","['6', '7']","['5', '10']","['5.74', '10.63']","['0.65', '2.13']"


# Remove Year from League Name
- unique comparable league names
- no duplicate years with date

In [38]:
data.league = data.league.apply(lambda x: x[:-4])

In [41]:
data.league.unique()

array(['EPL', 'Serie_A', 'Bundesliga', 'La_liga', 'Ligue_1', 'RFPL'],
      dtype=object)

# Convert string dates to python datetime objects

In [55]:
data.date = data.date.apply(lambda x: datetime.strptime(x, '%b %d %Y'))

In [56]:
data.date.describe()

count                            20980
mean     2019-05-20 21:14:47.473784320
min                2014-08-01 00:00:00
25%                2017-01-22 00:00:00
50%                2019-05-24 00:00:00
75%                2021-10-31 00:00:00
max                2023-12-03 00:00:00
Name: date, dtype: object

# Check team names
Minimum amount of name occurences should be amount of home matches in current season

In [58]:
data.team_away.value_counts()

team_away
Tottenham             207
West Ham              207
Liverpool             207
Manchester City       207
Manchester United     207
                     ... 
FC Rotor Volgograd     14
Baltika                 9
Luton                   8
FC Heidenheim           7
Le Havre                7
Name: count, Length: 191, dtype: int64

In [59]:
data.team_home.value_counts()

team_home
Arsenal               208
Manchester United     207
Everton               207
Chelsea               207
Tottenham             206
                     ... 
FC Rotor Volgograd     14
Baltika                 8
Le Havre                7
Luton                   6
FC Heidenheim           6
Name: count, Length: 191, dtype: int64

In [60]:
data.loc[data.team_home=='FC Heidenheim']

,id,league,date,team_home,team_away,chances,goals,xG,shots,shots_on_target,deep,ppda,xPTS
20761,23078,Bundesliga,2023-08-26,FC Heidenheim,Hoffenheim,"['37', '24', '39']","['2', '3']","['2.73', '2.76']","['20', '16']","['9', '6']","['8', '4']","['17.50', '6.44']","['1.34', '1.42']"
20782,23099,Bundesliga,2023-09-17,FC Heidenheim,Werder Bremen,"['23', '23', '54']","['4', '2']","['1.96', '2.56']","['18', '11']","['5', '4']","['1', '11']","['8.25', '9.05']","['0.91', '1.85']"
20795,23112,Bundesliga,2023-09-30,FC Heidenheim,Union Berlin,"['5', '15', '80']","['1', '0']","['0.54', '2.16']","['10', '20']","['4', '8']","['3', '9']","['16.60', '11.05']","['0.30', '2.56']"
20819,23136,Bundesliga,2023-10-22,FC Heidenheim,Augsburg,"['19', '22', '59']","['2', '5']","['1.99', '2.82']","['15', '9']","['5', '6']","['6', '4']","['16.14', '22.81']","['0.79', '1.99']"
20836,23153,Bundesliga,2023-11-05,FC Heidenheim,VfB Stuttgart,"['5', '11', '84']","['2', '0']","['0.78', '2.57']","['19', '16']","['3', '2']","['3', '14']","['25.43', '6.00']","['0.26', '2.63']"
20855,23172,Bundesliga,2023-11-26,FC Heidenheim,Bochum,"['44', '41', '15']","['0', '0']","['0.75', '0.36']","['9', '10']","['3', '2']","['4', '4']","['10.30', '12.16']","['1.72', '0.87']"


# Convert statistc values from string to float/int

In [74]:
eval(data.chances.loc[0])

['29', '39', '32']

In [76]:
data.chances = data.chances.apply(lambda x: [int(eval(x)[0]), int(eval(x)[1]), int(eval(x)[2])])

In [80]:
data.goals = data.goals.apply(lambda x: [int(eval(x)[0]), int(eval(x)[1])])

In [82]:
data.xG = data.xG.apply(lambda x: [float(eval(x)[0]), float(eval(x)[1])])

In [83]:
data.shots = data.shots.apply(lambda x: [int(eval(x)[0]), int(eval(x)[1])])

In [87]:
data.shots_on_target = data.shots_on_target.apply(lambda x: [int(eval(x)[0]), int(eval(x)[1])])

In [88]:
data.deep = data.deep.apply(lambda x: [int(eval(x)[0]), int(eval(x)[1])])

In [84]:
data.ppda = data.ppda.apply(lambda x: [float(eval(x)[0]), float(eval(x)[1])])

In [85]:
data.xPTS = data.xPTS.apply(lambda x: [float(eval(x)[0]), float(eval(x)[1])])

In [89]:
data.head()

,id,league,date,team_home,team_away,chances,goals,xG,shots,shots_on_target,deep,ppda,xPTS
0,81,EPL,2015-08-08,Manchester United,Tottenham,"[29, 39, 32]","[1, 0]","[0.63, 0.67]","[9, 9]","[1, 4]","[4, 10]","[13.83, 8.22]","[1.25, 1.36]"
1,82,EPL,2015-08-08,Bournemouth,Aston Villa,"[34, 36, 30]","[0, 1]","[0.88, 0.78]","[11, 7]","[2, 3]","[11, 2]","[6.9, 11.85]","[1.39, 1.25]"
2,83,EPL,2015-08-08,Everton,Watford,"[30, 43, 27]","[2, 2]","[0.6, 0.56]","[10, 11]","[5, 5]","[5, 4]","[6.65, 17.16]","[1.33, 1.24]"
3,84,EPL,2015-08-08,Leicester,Sunderland,"[64, 21, 15]","[4, 2]","[2.57, 1.46]","[19, 11]","[8, 5]","[5, 6]","[10.88, 9.56]","[2.13, 0.66]"
4,85,EPL,2015-08-08,Norwich,Crystal Palace,"[15, 21, 64]","[1, 3]","[1.13, 2.11]","[17, 11]","[6, 7]","[5, 10]","[5.74, 10.63]","[0.65, 2.13]"


In [91]:
write_file_path = git_root + '/data/standardized_data/understat_data_cleaned.csv'
data.to_csv(write_file_path)

# Data Validation TODO
Ideal would be a second dataset to verify the given data.\
Checks:\
- Consistent amount of matches per league / per team over the years